In [38]:
import json
from collections import OrderedDict

judge_file = 'judged_o3_mini_high.json'
response_file = 'response.json'

judge_datas = {v["Input"]:v for v in json.load(open(judge_file), object_pairs_hook=OrderedDict)}
response_datas = {v["Input"]:v for v in json.load(open(response_file), object_pairs_hook=OrderedDict)}

print(len(list(response_datas.keys())))

16


In [59]:
from pprint import pprint
import pandas as pd
import re

for k in judge_datas:
    judge_data = judge_datas[k]
    response_data = response_datas[k]
    # print(f'Input: {k}')
    # pprint(judge_data)
    
    ref, can = response_data['Reference'], response_data['Candidate']
    if type(can) == str:
        print(can)
        can_re = re.sub(
            r'\"timestamp\" BETWEEN \"(.*?)\" AND \"(.*?)\"\'', 
            r'"timestamp BETWEEN \1 AND \2"', 
        can)
        can_re = re.sub(r'("templates":\s*\[\s*)(.*?)(\s*\])', lambda m: m.group(1) + re.sub(r"'", '"', m.group(2)) + m.group(3), can_re, flags=re.DOTALL)
        can_re = re.sub('ngày', '', can_re)
        print(can_re)
        can = json.loads(can_re)
    
    ref_flat = pd.json_normalize(ref)
    can_flat = pd.json_normalize(can)
    
    # print(ref_flat)
    # print("\n\n------------")

{"Formalized Input": "이번주 우리반의 평균 실내온도 알려줘.", "Input Semantic Parsing": {"Temporal": {"이번주": "2022-09-26 00:00:00 ~ 2022-10-02 23:59:59"}, "Spatial": {"우리반": "01_IB5"}, "Modality": {"실내온도": "roomtemp"}, "Operation": ["평균"]}, "Strategy": {"Objective": "이번주 우리반의 실내온도 데이터 중 평균값을 계산함.", "Expected Outputs": ["이번주 우리반의 평균 온도는 25℃입니다.", "이번주 우리반의 평균 온도는 26℃입니다."], "Step": ["우리반의 이번주 실내온도 데이터를 쿼리해 qr_roomtemp_weekly에 저장", "qr_roomtemp_weekly에서 roomtemp의 평균값 계산", "평균값 반환"]}, "Instruction Set": [{"type": "q", "run": {"args": {"table_name": "data_t", "columns": ["roomtemp"], "conditions": ["timestamp" BETWEEN "2022-09-26 00:00:00" AND "2022-10-02 23:59:59"'], "subquery": "idu_id = (SELECT id FROM idu_t WHERE name = '01_IB5')"}}, "result": {"name": "qr_roomtemp_weekly"}}, {"type": "o", "python_script_to_process": "avg_roomtemp = qr_roomtemp_weekly['roomtemp'].mean()", "return": ["avg_roomtemp"]}, {"type": "r", "templates": ["이번주 우리반의 평균 온도는 {avg_roomtemp}℃입니다.", "이번주 우리반의 평균 온도는 {avg_roomtemp}℃입니다

JSONDecodeError: Expecting ',' delimiter: line 1 column 1029 (char 1028)